In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [91]:
df=pd.read_csv(r'c:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\J1 2024_players.csv')

In [92]:

# Define the position mapping
position_to_class = {
    "LCMF3": "Central Midfielders", "RCMF3": "Central Midfielders",
    "LAMF": "Wingers", "LW": "Wingers", "RB": "FBs & WBs", "LB": "FBs & WBs",
    "LCMF": "Central Midfielders", "DMF": "Defensive Midfielders", 
    "RDMF": "Defensive Midfielders", "RWF": "Wingers", "AMF": "Attacking Midfielders",
    "LCB": "Center Backs", "RWB": "FBs & WBs", "CF": "Strikers", 
    "LWB": "FBs & WBs", "GK": "Goalkeepers", "LDMF": "Defensive Midfielders",
    "RCMF": "Central Midfielders", "LWF": "Wingers", "RW": "Wingers", 
    "RAMF": "Wingers", "RCB": "Center Backs", "CB": "Center Backs", 
    "RCB3": "Center Backs", "LCB3": "Center Backs", "RB5": "FBs & WBs", 
    "RWB5": "FBs & WBs", "LB5": "FBs & WBs", "LWB5": "FBs & WBs"
}


df["position_category"] = df["Primary position"].map(position_to_class)

df["position_category"].fillna("Unknown", inplace=True)

df.to_csv(r'c:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\J1 2024_players_classified.csv', index=False)



C:\Users\lahd2\AppData\Local\Temp\ipykernel_39720\3791741993.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["position_category"].fillna("Unknown", inplace=True)


In [103]:
# Reload data and prepare for full position analysis
df = df

# Map positions to categories
position_to_class = {
    "LCMF3": "Central Midfielders", "RCMF3": "Central Midfielders",
    "LAMF": "Wingers", "LW": "Wingers", "RB": "FBs & WBs", "LB": "FBs & WBs",
    "LCMF": "Central Midfielders", "DMF": "Defensive Midfielders", 
    "RDMF": "Defensive Midfielders", "RWF": "Wingers", "AMF": "Attacking Midfielders",
    "LCB": "Center Backs", "RWB": "FBs & WBs", "CF": "Strikers", 
    "LWB": "FBs & WBs", "GK": "Goalkeepers", "LDMF": "Defensive Midfielders",
    "RCMF": "Central Midfielders", "LWF": "Wingers", "RW": "Wingers", 
    "RAMF": "Wingers", "RCB": "Center Backs", "CB": "Center Backs", 
    "RCB3": "Center Backs", "LCB3": "Center Backs", "RB5": "FBs & WBs", 
    "RWB5": "FBs & WBs", "LB5": "FBs & WBs", "LWB5": "FBs & WBs"
}

# Apply mapping
df["position_category"] = df["Primary position"].map(position_to_class)

# Drop unmapped rows
df = df.dropna(subset=["position_category"])

# Filter for players with more than 20 matches
df = df[df["Matches played"] > 20]

# Numeric columns minus the excluded ones
exclude = [
    "id", "Wyscout id", "teamId", "Minutes played", "Matches played",
    "Contract expires", "Market value", "Birthday", "Age", "Height", "Weight",
    "Primary position, %", "Secondary position, %", "Third position, %"
]
numeric_cols = df.select_dtypes(include=np.number).columns.difference(exclude).tolist()

# Filter columns based on relevance across position categories (≥ 20% non-zero in any group)
valid_cols = []
for col in numeric_cols:
    non_zero_ratio = df.groupby("position_category")[col].apply(lambda x: (x != 0).sum() / len(x))
    if (non_zero_ratio > 0.2).any():
        valid_cols.append(col)

# Recalculate group mean and identify relevant features
group_means = df.groupby("position_category")[valid_cols].mean()
overall_mean = df[valid_cols].mean()
ratio = group_means / overall_mean
relevance_score = abs(ratio - 1)
important_features = (relevance_score > 0.5)

# Create dict of relevant columns per position
columns_by_position = {}
for position in group_means.index:
    relevant = important_features.loc[position]
    columns_by_position[position] = relevant[relevant].index.tolist()

# Convert to DataFrame for display
position_feature_df_filtered = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in columns_by_position.items()]))



position_feature_df_filtered.to_csv(r"c:\Users\lahd2\OneDrive\Mehdaf\Ai_comp\position_feature_df.csv")


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load your player dataset
df = pd.read_csv("J1 2024_players_classified.csv")

# Define the clustering metrics for each position category
position_category_metrics = {
    "Goalkeepers": ["Forward passes per 90", "Long passes per 90", "Average pass length, m",
                    "Average long pass length, m", "Progressive passes per 90", "Vertical passes per 90",
                    "Conceded goals per 90", "Shots against per 90", "Clean sheets", "Save rate, %",
                    "xG against per 90", "Prevented goals per 90", "Back passes received as GK per 90", "Exits per 90"],

    "Center Backs": ["Duels per 90", "Successful defensive actions per 90", "Defensive duels per 90",
                     "Aerial duels per 90", "Shots blocked per 90", "Interceptions per 90", "PAdj Interceptions",
                     "Forward passes per 90", "Short / medium passes per 90", "Long passes per 90",
                     "Shot assists per 90", "Passes to final third per 90", "Through passes per 90",
                     "Progressive passes per 90", "Vertical passes per 90"],

    "FBs & WBs": ["Aerial duels per 90", "Duels per 90", "Successful defensive actions per 90", "Interceptions per 90",
                  "Successful attacking actions per 90", "Shots per 90", "Assists per 90", "Crosses per 90",
                  "Crosses to goalie box per 90", "Dribbles per 90", "Progressive runs per 90", "Accelerations per 90",
                  "Passes per 90", "Forward passes per 90", "Progressive passes per 90", "xA per 90", "Key passes per 90"],

    "Defensive Midfielders": ["Duels per 90", "Successful defensive actions per 90", "Defensive duels per 90",
                              "Aerial duels per 90", "Sliding tackles per 90", "Interceptions per 90", "PAdj Interceptions",
                              "Passes per 90", "Forward passes per 90", "Back passes per 90", "Vertical passes per 90",
                              "Through passes per 90", "Progressive passes per 90", "Smart passes per 90"],

    "Central Midfielders": ["Goals", "Assists", "xA", "Sliding tackles per 90", "PAdj Sliding tackles", "Interceptions per 90",
                            "PAdj Interceptions", "Shots per 90", "Crosses per 90", "Received passes per 90",
                            "Passes per 90", "Forward passes per 90", "Back passes per 90", "Through passes per 90",
                            "Key passes per 90", "Progressive passes per 90"],

    "Attacking Midfielders": ["Goals", "xG", "Assists", "xA", "Goals per 90", "Shots per 90", "Goal conversion, %",
                              "Assists per 90", "Dribbles per 90", "Touches in box per 90", "Progressive runs per 90",
                              "Accelerations per 90", "xA per 90", "Shot assists per 90", "Smart passes per 90",
                              "Key passes per 90", "Passes to penalty area per 90"],

    "Wingers": ["Goals", "xG", "Assists", "xA", "Duels per 90", "Successful attacking actions per 90",
                "Shots per 90", "Crosses per 90", "Dribbles per 90", "Touches in box per 90",
                "Progressive runs per 90", "Accelerations per 90", "xA per 90", "Smart passes per 90",
                "Deep completions per 90", "Key passes per 90"],

    "Strikers": ["Goals", "xG", "Goals per 90", "Non-penalty goals", "xG per 90", "Shots per 90",
                 "Goal conversion, %", "Head goals", "Dribbles per 90", "Touches in box per 90",
                 "Progressive runs per 90", "Accelerations per 90", "xA per 90", "Shot assists per 90",
                 "Smart passes per 90", "Key passes per 90", "Passes to penalty area per 90"]
}

# Style names (fixed for 3 clusters per category)
style_name_map = {
    "Goalkeepers": ["Ball-Playing Keeper", "Sweeper Keeper", "Line Keeper"],
    "Center Backs": ["Ball-Playing Defender", "Stopper", "Sweeper (Libero)"],
    "FBs & WBs": ["Wing Back (Offensive)", "Full Back (Defensive)", "Inverted Wing Back"],
    "Defensive Midfielders": ["Holding Midfielder", "Ball-Winning Midfielder", "Deep Lying Playmaker"],
    "Central Midfielders": ["Box to Box", "Wide Midfielder (Mezzala)", "Deep Lying Playmaker"],
    "Attacking Midfielders": ["Advanced Playmaker", "Trequartista", "False Attacking Midfielder"],
    "Wingers": ["Inside Forward", "Wide Playmaker", "Winger"],
    "Strikers": ["Poacher", "False Nine", "Target Man"]
}

clustered_results = []

for category, metrics in position_category_metrics.items():
    df_cat = df[df["position_category"] == category].dropna(subset=metrics).copy()
    if df_cat.empty or len(df_cat) < 3:
        continue

    X = df_cat[metrics]
    X_scaled = StandardScaler().fit_transform(X)
    kmeans = KMeans(n_clusters=3, random_state=42)
    labels = kmeans.fit_predict(X_scaled)

    style_labels = style_name_map[category]
    df_cat["player_style"] = [style_labels[i] for i in labels]

    clustered_results.append(df_cat[["Player", "player_style"]])

# Merge player styles back
style_df = pd.concat(clustered_results, ignore_index=True)
df_clustered = df.merge(style_df, on="Player", how="left")


In [ ]:
df_clustered["non_null_count"] = df_clustered.notnull().sum(axis=1)

df_summed = df_clustered.sort_values("non_null_count", ascending=False)
df_summed = df_summed.drop_duplicates(subset=["Player"], keep="first")
df_summed.drop(columns=["non_null_count"], inplace=True)

df_summed.to_csv("J1_2024_Player_Styles_Descriptive_Summed.csv", index=False)


In [ ]:


# Load clustered and summed player data
df = pd.read_csv("J1_2024_Player_Styles_Descriptive_Summed.csv")

# Position-specific metric definitions (same as clustering phase)
position_category_metrics = {
    "Goalkeepers": ["Forward passes per 90", "Long passes per 90", "Average pass length, m",
                    "Average long pass length, m", "Progressive passes per 90", "Vertical passes per 90",
                    "Conceded goals per 90", "Shots against per 90", "Clean sheets", "Save rate, %",
                    "xG against per 90", "Prevented goals per 90", "Back passes received as GK per 90", "Exits per 90"],

    "Center Backs": ["Duels per 90", "Successful defensive actions per 90", "Defensive duels per 90",
                     "Aerial duels per 90", "Shots blocked per 90", "Interceptions per 90", "PAdj Interceptions",
                     "Forward passes per 90", "Short / medium passes per 90", "Long passes per 90",
                     "Shot assists per 90", "Passes to final third per 90", "Through passes per 90",
                     "Progressive passes per 90", "Vertical passes per 90"],

    "FBs & WBs": ["Aerial duels per 90", "Duels per 90", "Successful defensive actions per 90", "Interceptions per 90",
                  "Successful attacking actions per 90", "Shots per 90", "Assists per 90", "Crosses per 90",
                  "Crosses to goalie box per 90", "Dribbles per 90", "Progressive runs per 90", "Accelerations per 90",
                  "Passes per 90", "Forward passes per 90", "Progressive passes per 90", "xA per 90", "Key passes per 90"],

    "Defensive Midfielders": ["Duels per 90", "Successful defensive actions per 90", "Defensive duels per 90",
                              "Aerial duels per 90", "Sliding tackles per 90", "Interceptions per 90", "PAdj Interceptions",
                              "Passes per 90", "Forward passes per 90", "Back passes per 90", "Vertical passes per 90",
                              "Through passes per 90", "Progressive passes per 90", "Smart passes per 90"],

    "Central Midfielders": ["Goals", "Assists", "xA", "Sliding tackles per 90", "PAdj Sliding tackles", "Interceptions per 90",
                            "PAdj Interceptions", "Shots per 90", "Crosses per 90", "Received passes per 90",
                            "Passes per 90", "Forward passes per 90", "Back passes per 90", "Through passes per 90",
                            "Key passes per 90", "Progressive passes per 90"],

    "Attacking Midfielders": ["Goals", "xG", "Assists", "xA", "Goals per 90", "Shots per 90", "Goal conversion, %",
                              "Assists per 90", "Dribbles per 90", "Touches in box per 90", "Progressive runs per 90",
                              "Accelerations per 90", "xA per 90", "Shot assists per 90", "Smart passes per 90",
                              "Key passes per 90", "Passes to penalty area per 90"],

    "Wingers": ["Goals", "xG", "Assists", "xA", "Duels per 90", "Successful attacking actions per 90",
                "Shots per 90", "Crosses per 90", "Dribbles per 90", "Touches in box per 90",
                "Progressive runs per 90", "Accelerations per 90", "xA per 90", "Smart passes per 90",
                "Deep completions per 90", "Key passes per 90"],

    "Strikers": ["Goals", "xG", "Goals per 90", "Non-penalty goals", "xG per 90", "Shots per 90",
                 "Goal conversion, %", "Head goals", "Dribbles per 90", "Touches in box per 90",
                 "Progressive runs per 90", "Accelerations per 90", "xA per 90", "Shot assists per 90",
                 "Smart passes per 90", "Key passes per 90", "Passes to penalty area per 90"]
}


# Generate PCA plot per category
for category, metrics in position_category_metrics.items():
    df_cat = df[df["position_category"] == category].dropna(subset=metrics)
    if df_cat.empty or df_cat["player_style"].nunique() < 2:
        continue

    # Standardize data
    X = df_cat[metrics]
    X_scaled = StandardScaler().fit_transform(X)

    # Apply PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(X_scaled)

    df_cat["PCA1"] = pca_result[:, 0]
    df_cat["PCA2"] = pca_result[:, 1]

    # Plot
    plt.figure(figsize=(10, 7))
    sns.scatterplot(
        data=df_cat,
        x="PCA1", y="PCA2",
        hue="player_style",
        palette="Set2",
        s=80,
        edgecolor="black"
    )
    plt.title(f"PCA Scatter Plot: {category} Styles")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
